This is for running on kaggle 

In [ ]:
!pip download cmake -d ./cmake/

In [ ]:
!cmake --version

In [ ]:
!git clone https://github.com/dudegladiator/ML-CO-Policy-for-VRPTW

In [ ]:
!pip install -r /kaggle/working/ML-CO-Policy-for-VRPTW/requirements.txt

In [ ]:
%cd /kaggle/working/ML-CO-Policy-for-VRPTW/pchgs

In [ ]:
!pwd

In [ ]:
!mkdir build
%cd build
!cmake .. -DCMAKE_BUILD_TYPE="RELEASE"

In [ ]:
!make

# Training

In [ ]:
%cd /kaggle/working/ML-CO-Policy-for-VRPTW/training

In [ ]:
!pwd

In [ ]:
import os
import subprocess

def run_experiment(job_name, DIR_ORACLE, DIR_MODELS, NUM_PERTURBATIONS, SD_PERTURBATION, TIME_LIMIT, LEARNING_RATE, PREDICTOR, FEATURE_SET):
    cmd = [
        'python3', 'run_training.py',
        '--oracle_solutions_directory', DIR_ORACLE,
        '--dir_models', DIR_MODELS,
        '--num_perturbations', str(NUM_PERTURBATIONS),
        '--sd_perturbation', str(SD_PERTURBATION),
        '--time_limit', str(TIME_LIMIT),
        '--learning_rate', str(LEARNING_RATE),
        '--predictor', PREDICTOR,
        '--feature_set', FEATURE_SET
    ]
    subprocess.run(cmd, check=True)

In [ ]:
import os
import subprocess

DIR_MODELS = "./models/"
NUM_PERTURBATIONS = 20
SD_PERTURBATION = 1
LEARNING_RATE = 0.01
TIME_LIMIT = 90
FEATURE_SET = "dynamicstatic"
PREDICTOR = "NeuralNetwork"
SAMPLESIZE = 50
NUMINSTANCES = 15
RUNTIMESTRATEGY = 3600


# BASE CASE
DIR_ORACLE = f"../experiments/samples-{SAMPLESIZE}_instances-{NUMINSTANCES}_runtime-{RUNTIMESTRATEGY}/"
print("basecase is runnning")
run_experiment("base", DIR_ORACLE, DIR_MODELS, NUM_PERTURBATIONS, SD_PERTURBATION, TIME_LIMIT, LEARNING_RATE, PREDICTOR, FEATURE_SET)

# EXPERIMENT 1
for SAMPLESIZE in [10, 25, 75, 100]:
    DIR_ORACLE = f"../experiments/samples-{SAMPLESIZE}_instances-{NUMINSTANCES}_runtime-{RUNTIMESTRATEGY}/"
    print("experiment1")
    run_experiment("exp1", DIR_ORACLE, DIR_MODELS, NUM_PERTURBATIONS, SD_PERTURBATION, TIME_LIMIT, LEARNING_RATE, PREDICTOR, FEATURE_SET)

# EXPERIMENT 2
SAMPLESIZE = 50
for NUMINSTANCES in [1, 2, 5, 10, 20, 25, 30]:
    DIR_ORACLE = f"../experiments/samples-{SAMPLESIZE}_instances-{NUMINSTANCES}_runtime-{RUNTIMESTRATEGY}/"
    print("experiment2")
    run_experiment("exp2", DIR_ORACLE, DIR_MODELS, NUM_PERTURBATIONS, SD_PERTURBATION, TIME_LIMIT, LEARNING_RATE, PREDICTOR, FEATURE_SET)

# EXPERIMENT 3
NUMINSTANCES = 15
for RUNTIMESTRATEGY in [60, 120, 180, 240, 300, 900, "bestSeed"]:
    DIR_ORACLE = f"../experiments/samples-{SAMPLESIZE}_instances-{NUMINSTANCES}_runtime-{RUNTIMESTRATEGY}/"
    print("experiment3")
    run_experiment("exp3", DIR_ORACLE, DIR_MODELS, NUM_PERTURBATIONS, SD_PERTURBATION, TIME_LIMIT, LEARNING_RATE, PREDICTOR, FEATURE_SET)

# EXPERIMENT 4
RUNTIMESTRATEGY = 3600
for FEATURE_SET in ["dynamic", "static"]:
    DIR_ORACLE = f"../experiments/samples-{SAMPLESIZE}_instances-{NUMINSTANCES}_runtime-{RUNTIMESTRATEGY}/"
    print("experiment4")
    run_experiment("exp4", DIR_ORACLE, DIR_MODELS, NUM_PERTURBATIONS, SD_PERTURBATION, TIME_LIMIT, LEARNING_RATE, PREDICTOR, FEATURE_SET)

# EXPERIMENT 5
for PREDICTOR in ["GraphNeuralNetwork", "Linear", "GraphNeuralNetwork_sparse"]:
    DIR_ORACLE = f"../experiments/samples-{SAMPLESIZE}_instances-{NUMINSTANCES}_runtime-{RUNTIMESTRATEGY}/"
    print("experiment5")
    run_experiment("exp5", DIR_ORACLE, DIR_MODELS, NUM_PERTURBATIONS, SD_PERTURBATION, TIME_LIMIT, LEARNING_RATE, PREDICTOR, FEATURE_SET)

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

def plot_learning_curves(loss_curve, accuracy_curve_n, default_accuracy_n, title=None):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(list(range(len(loss_curve))), loss_curve)
    ax1.set_title("Loss")
    ax1.set_ylim(bottom=0, top=max(np.array(loss_curve)) + 100)
    ax2.plot(list(range(len(accuracy_curve_n))), accuracy_curve_n)
    ax2.hlines(default_accuracy_n, xmin=0, xmax=len(loss_curve), colors="red")
    ax2.set_title("Accuracy Nodes")
    if title is not None:
        fig.suptitle(title + "\n" + "\n" + "(The red line specifies the default accuracy)", fontsize=7)
    fig.supxlabel('training epochs')
    plt.tight_layout()
    plt.show()


def get_learning_curve(learning_name):
    parameter_evolution = json.load(open('{}'.format(learning_name), 'r'))
    plot_learning_curves(loss_curve=parameter_evolution["overall_loss"],
                         accuracy_curve_n=parameter_evolution["overall_accuracy_n"],
                         default_accuracy_n=parameter_evolution["default_accuracy_n"],
                         title=learning_name)
    
# Please enter the learning curve you want to plot
get_learning_curve("/kaggle/working/ML-CO-Policy-for-VRPTW/training/learning_evaluation_NeuralNetwork_samples-test_instances-test_runtime-test_featureset-dynamicstatic.json")

# Validation

In [ ]:
%cd /kaggle/working/ML-CO-Policy-for-VRPTW/evaluation

In [ ]:
import os
import subprocess

def run_solve_bound(strategy, model_name, instance_directory, time_limit, result_directory, instance_seed, dir_models):
    cmd = [
        'python3', 'solve_bound.py',
        '--strategy', strategy,
        '--model_name', model_name,
        '--instance_directory', instance_directory,
        '--time_limit', str(time_limit),
        '--result_directory', result_directory,
        '--instance_seed', str(instance_seed),
        '--monte_carlo_sampling_rounds', '9',
        '--model_directory', dir_models
    ]
    subprocess.run(cmd, check=True)

In [ ]:
def main():
    time_limit = 90
    instance_directory = "./validation_instances/"
    strategy = "NeuralNetwork"
    result_directory = "./results/results_validation/"
    instance_seeds = ["1000"]
    dir_models = "../training/models/"
    feature_set = "dynamicstatic"
    sample_size = 50
    num_instances = 15
    runtime_strategy = 3600
    training_iterations = [29]

    # BASE CASE
    model_name_base = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{{}}"
    for instance_seed in instance_seeds:
        for iteration in training_iterations:
            run_solve_bound(strategy, model_name_base.format(iteration), instance_directory, time_limit, result_directory, instance_seed, dir_models)

    # EXPERIMENT 1
    for sample_size in [10, 25, 75, 100]:
        model_name_exp1 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{{}}"
        for instance_seed in instance_seeds:
            for iteration in training_iterations:
                run_solve_bound(strategy, model_name_exp1.format(iteration), instance_directory, time_limit, result_directory, instance_seed, dir_models)

    # EXPERIMENT 2
    sample_size = 50
    for num_instances in [1, 2, 5, 10, 20, 25, 30]:
        model_name_exp2 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{{}}"
        for instance_seed in instance_seeds:
            for iteration in training_iterations:
                run_solve_bound(strategy, model_name_exp2.format(iteration), instance_directory, time_limit, result_directory, instance_seed, dir_models)

    # EXPERIMENT 3
    num_instances = 15
    for runtime_strategy in [60, 120, 180, 240, 300, 900, "bestSeed"]:
        model_name_exp3 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{{}}"
        for instance_seed in instance_seeds:
            for iteration in training_iterations:
                run_solve_bound(strategy, model_name_exp3.format(iteration), instance_directory, time_limit, result_directory, instance_seed, dir_models)

    # EXPERIMENT 4
    runtime_strategy = 3600
    for feature_set in ["static", "dynamic"]:
        model_name_exp4 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{{}}"
        for instance_seed in instance_seeds:
            for iteration in training_iterations:
                run_solve_bound(strategy, model_name_exp4.format(iteration), instance_directory, time_limit, result_directory, instance_seed, dir_models)

    # EXPERIMENT 5
    feature_set = "dynamicstatic"
    for strategy in ["GraphNeuralNetwork", "Linear", "GraphNeuralNetwork_sparse"]:
        model_name_exp5 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{{}}"
        for instance_seed in instance_seeds:
            for iteration in training_iterations:
                run_solve_bound(strategy, model_name_exp5.format(iteration), instance_directory, time_limit, result_directory, instance_seed, dir_models)

main()

# Testing

In [ ]:
%cd /kaggle/working/ML-CO-Policy-for-VRPTW/evaluation

In [ ]:
import os
import subprocess

def run_solve_bound(strategy, model_name, instance_directory, time_limit, result_directory, instance_seed, dir_models, monte_carlo_sampling_rounds=9):
    cmd = [
        'python3', 'solve_bound.py',
        '--strategy', strategy,
        '--model_name', model_name,
        '--instance_directory', instance_directory,
        '--time_limit', str(time_limit),
        '--result_directory', result_directory,
        '--instance_seed', str(instance_seed),
        '--monte_carlo_sampling_rounds', str(monte_carlo_sampling_rounds),
        '--model_directory', dir_models
    ]
    subprocess.run(cmd, check=True)


Write the best model from validation, here

In [ ]:
array_learning_iteration = {
    "NeuralNetwork_samples-50_instances-15_runtime-3600_featureset-dynamicstatic":  "29",
    "NeuralNetwork_samples-10_instances-15_runtime-3600_featureset-dynamicstatic": "30",
    "NeuralNetwork_samples-75_instances-15_runtime-3600_featureset-dynamicstatic": "30",
    "NeuralNetwork_samples-25_instances-15_runtime-3600_featureset-dynamicstatic": "12",
    "NeuralNetwork_samples-100_instances-15_runtime-3600_featureset-dynamicstatic": "24",
    "NeuralNetwork_samples-50_instances-30_runtime-3600_featureset-dynamicstatic": "11",
    "NeuralNetwork_samples-50_instances-2_runtime-3600_featureset-dynamicstatic": "18",
    "NeuralNetwork_samples-50_instances-10_runtime-3600_featureset-dynamicstatic": "30",
    "NeuralNetwork_samples-50_instances-1_runtime-3600_featureset-dynamicstatic": "19",
    "NeuralNetwork_samples-50_instances-25_runtime-3600_featureset-dynamicstatic": "19",
    "NeuralNetwork_samples-50_instances-5_runtime-3600_featureset-dynamicstatic": "23",
    "NeuralNetwork_samples-50_instances-20_runtime-3600_featureset-dynamicstatic": "24",
    "NeuralNetwork_samples-50_instances-15_runtime-240_featureset-dynamicstatic": "27",
    "NeuralNetwork_samples-50_instances-15_runtime-bestSeed_featureset-dynamicstatic": "18",
    "NeuralNetwork_samples-50_instances-15_runtime-180_featureset-dynamicstatic": "15",
    "NeuralNetwork_samples-50_instances-15_runtime-900_featureset-dynamicstatic": "25",
    "NeuralNetwork_samples-50_instances-15_runtime-60_featureset-dynamicstatic": "23",
    "NeuralNetwork_samples-50_instances-15_runtime-300_featureset-dynamicstatic": "30",
    "NeuralNetwork_samples-50_instances-15_runtime-120_featureset-dynamicstatic": "30",
    "NeuralNetwork_samples-50_instances-15_runtime-3600_featureset-dynamic": "23",
    "NeuralNetwork_samples-50_instances-15_runtime-3600_featureset-static": "1",
    "GraphNeuralNetwork_sparse_samples-50_instances-15_runtime-3600_featureset-dynamicstatic": "21",
    "Linear_samples-50_instances-15_runtime-3600_featureset-dynamicstatic": "28",
    "GraphNeuralNetwork_samples-50_instances-15_runtime-3600_featureset-dynamicstatic": "22"
}

In [ ]:
def main():
    time_limit = 90
    instance_directory = "./test_instances/"
    strategy = "NeuralNetwork"
    result_directory = "./results/results_test/"
    instance_seeds = [int(i) for i in "10031 10032 10033 10034 10035 10036 10037 10038 10039 10040 10041 10042 10043 10044 10045 10046 10047 10048 10049 10050 10001 10002 10003 10004 10005 10006 10007 10008 10009 10010 10011 10012 10013 10014 10015 10016 10017 10018 10019 10020 10021 10022 10023 10024 10025 10026 10027 10028 10029 10030".split()]
    dir_models = "../training/models/"
    feature_set = "dynamicstatic"
    sample_size = 50
    num_instances = 15
    runtime_strategy = 3600

    # Reading learning iterations from the file
#     array_learning_iteration = {}
#     with open("./src/read_in_learning_iteration.txt") as f:
#         lines = f.readlines()
#         for line in lines:
#             key, value = line.strip().split('=')
#             array_learning_iteration[key] = value

    # BASE CASE
    iteration = array_learning_iteration[f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}"]
    model_name_base = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{iteration}"
    for instance_seed in instance_seeds:
        run_solve_bound(strategy, model_name_base, instance_directory, time_limit, result_directory, instance_seed, dir_models)

    # EXPERIMENT 0
    for strategy in ["rolling_horizon", "monte_carlo", "oracle", "greedy", "random", "lazy"]:
        if strategy == "oracle":
            time_limit = 3600
        elif strategy in ["monte_carlo", "rolling_horizon"]:
            time_limit = 600
        else:
            time_limit = 90
        iteration = -1
        model_name_exp0 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{iteration}"
        for instance_seed in instance_seeds:
            run_solve_bound(strategy, model_name_exp0, instance_directory, time_limit, result_directory, instance_seed, dir_models)

    # Reset strategy and time limit
    strategy = "NeuralNetwork"
    time_limit = 90

    # EXPERIMENT 1
    for sample_size in [10, 25, 75, 100]:
        iteration = array_learning_iteration[f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}"]
        model_name_exp1 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{iteration}"
        for instance_seed in instance_seeds:
            run_solve_bound(strategy, model_name_exp1, instance_directory, time_limit, result_directory, instance_seed, dir_models)

    sample_size = 50

    # EXPERIMENT 2
    for num_instances in [1, 2, 5, 10, 20, 25, 30]:
        iteration = array_learning_iteration[f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}"]
        model_name_exp2 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{iteration}"
        for instance_seed in instance_seeds:
            run_solve_bound(strategy, model_name_exp2, instance_directory, time_limit, result_directory, instance_seed, dir_models)

    num_instances = 15

    # EXPERIMENT 3
    for runtime_strategy in [60, 120, 180, 240, 300, 900, "bestSeed"]:
        iteration = array_learning_iteration[f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}"]
        model_name_exp3 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{iteration}"
        for instance_seed in instance_seeds:
            run_solve_bound(strategy, model_name_exp3, instance_directory, time_limit, result_directory, instance_seed, dir_models)

    runtime_strategy = 3600

    # EXPERIMENT 4
    for feature_set in ["static", "dynamic"]:
        iteration = array_learning_iteration[f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}"]
        model_name_exp4 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{iteration}"
        for instance_seed in instance_seeds:
            run_solve_bound(strategy, model_name_exp4, instance_directory, time_limit, result_directory, instance_seed, dir_models)

    feature_set = "dynamicstatic"

    # EXPERIMENT 5
    for strategy in ["GraphNeuralNetwork", "Linear", "GraphNeuralNetwork_sparse"]:
        iteration = array_learning_iteration[f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}"]
        model_name_exp5 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{iteration}"
        for instance_seed in instance_seeds:
            run_solve_bound(strategy, model_name_exp5, instance_directory, time_limit, result_directory, instance_seed, dir_models)

    strategy = "NeuralNetwork"

    # EXPERIMENT 6
    for runtime in [30, 60, 120, 180, 240]:
        iteration = array_learning_iteration[f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}"]
        model_name_exp6 = f"{strategy}_samples-{sample_size}_instances-{num_instances}_runtime-{runtime_strategy}_featureset-{feature_set}_iteration-{iteration}"
        for instance_seed in instance_seeds:
            run_solve_bound(strategy, model_name_exp6, instance_directory, runtime, result_directory, instance_seed, dir_models)


main()